In [ ]:
%reset -f

&nbsp;

---

Аналогія: код ("тіло") функції vs ім'я функції:

In [ ]:
def h(x):
  print('x =', x)
k = h
k(5)

x = 5


&nbsp;

---

Динамічне ("на льоту") створення класу:

In [ ]:
def f1(slf):
  print('func1 called!')
  res = f"Hello, my name is {slf.name}!"
  return res

def g2(slf, word):
  print('initializing, word =', word)
  slf.name = word

attrs = {
    'greet': f1,
    '__init__': g2
}

C1 = type('Person', (object,), attrs)

In [ ]:
# Create an instance of the dynamically created class
obj = C1("Alice")

initializing, word = Alice


In [ ]:
print(type(obj))

<class '__main__.Person'>


  Бачимо, що друкує не 'C1', тобто "суть" класу була "поміщена" у змінну `C1`, хоча ім'я клас має 'Person'.

In [ ]:
isinstance(obj, C1)

True

Перевіримо, що `obj` дійсно працює:

In [ ]:
msg = obj.greet()
print('msg:', msg)

func1 called!
msg: Hello, my name is Alice!


Bonus: "ручне" створення об'єкта ("вручну" можна створювати не лише класи!)

In [ ]:
x = object.__new__(C1)

In [ ]:
isinstance(x, C1)

True

In [ ]:
x.greet()

func1 called!


AttributeError: 'Person' object has no attribute 'name'

In [ ]:
x.__init__('Bob')

initializing, word = Bob


In [ ]:
x.greet()

func1 called!


'Hello, my name is Bob!'

&nbsp;

---

Деякі спеціальні методи, що стосуються метакласів:

* для класу:

`__prepare__`: Called before `__new__` to create the namespace dictionary for the class.

`__new__`: Creates and returns the class object.

`__init__`: Initializes the newly created class.

* для об'єктів:

`__call__`: Called when creating an instance of the class.

`__instancecheck__`: Used for isinstance() checks.

`__subclasscheck__`: Used for issubclass() checks.

* Інші атрибути метакласа:

Class attributes and methods defined in a metaclass become attributes of the type object of the classes created with that metaclass. They are accessible on the class itself, but not directly on instances of the class.

In [ ]:
class MyMetaclass(type):
    def __prepare__(name, bases):
        print("__prepare__ called")
        return {'prepared_attr': 'bonus!'}

    def __new__(mcs, name, bases, namespace):
        res = super().__new__(mcs, name, bases, namespace)
        print("__new__ called --->", res)
        return res

    def __init__(cls, name, bases, namespace):
        res = super().__init__(name, bases, namespace)
        print("__init__ called --->", res)
        return res

    def __call__(cls, *args, **kwargs):
        res = super().__call__(*args, **kwargs)
        print("__call__ called --->", res)
        return res

    def metaclass_method(cls):
        print("metaclass_method called")

    class_attribute = "I'm a class attribute of the metaclass"



Примітки:

It's generally recommended to inherit from `type` when creating a metaclass, but it's not strictly necessary in all cases (If you don't inherit from `type`, you'd need to implement all the necessary methods to create and initialize classes yourself, which is complex and error-prone)

Calling `super()` in metaclass methods is a good practice, but not always strictly necessary.

In [ ]:
class MyClass(metaclass=MyMetaclass):
    pass

__prepare__ called
__new__ called ---> <class '__main__.MyClass'>
__init__ called ---> None


In [ ]:
obj = MyClass()

__call__ called ---> <__main__.MyClass object at 0x79b0bda52230>


In [ ]:
dir(obj) # має prepared_attr, але не має metaclass_method та class_attribute

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'prepared_attr']

In [ ]:
print(obj.prepared_attr)

bonus!


In [ ]:
print(type(MyClass))

<class '__main__.MyMetaclass'>


&nbsp;

---

Як (і куди) "успадковуються" інші атрибути метакласа

In [ ]:
# Attributes and methods defined in a metaclass do not automatically become attributes or methods of the classes created by that metaclass.
# Instead, they are available on the class object itself, but not on instances of the class.
print(type(MyClass).metaclass_method)
print(type(MyClass).class_attribute)

<function MyMetaclass.metaclass_method at 0x79b0a4185bd0>
I'm a class attribute of the metaclass


In [ ]:
# Demonstrating metaclass method
MyClass.metaclass_method()

metaclass_method called


In [ ]:
# Calling from instance:
obj.metaclass_method()

AttributeError: 'MyClass' object has no attribute 'metaclass_method'

In [ ]:
# Proving it's a class method, not an instance method
print(type(MyClass.metaclass_method))

<class 'method'>


In [ ]:
# Demonstrating class attribute
print(MyClass.class_attribute)

I'm a class attribute of the metaclass


In [ ]:
# Accessing class attribute from instance:
print(obj.class_attribute)

AttributeError: 'MyClass' object has no attribute 'class_attribute'

&nbsp;

---

Приклад: "антиабстрактний клас" -- клас, в якому є метод, який заборонено заміщувати в класах-нащадках

In [ ]:
class NoOverrideMeta(type):
    def __init__(cls, name, bases, dct):
        # .........................................

In [ ]:
# Базовий клас з метакласом
class BaseClass(metaclass=NoOverrideMeta):
    def fixed(self):
        print("Привіт від метода, який не можна перевизначати!")

    def regular_method(self):
        print("Цей метод можна перевизначити.")


In [ ]:
class DerivedClass(BaseClass):
    def regular_method(self):
        print("Цей метод був перевизначений.")


In [ ]:
# Приклад використання
base = BaseClass()
base.fixed()

Привіт від метода, який не можна перевизначати!


In [ ]:
derived = DerivedClass()
derived.fixed()
derived.regular_method()

Привіт від метода, який не можна перевизначати!
Цей метод був перевизначений.


In [ ]:
class DerivedClass(BaseClass):
    # Це спричинить помилку при створенні класу
    def fixed(self):
         print("Спроба!")

    def regular_method(self):
        print("Цей2")


TypeError: Метод 'fixed' не можна перевизначати в класі 'DerivedClass'

&nbsp;

---

In [ ]:
class LimitedInstances(type):
    _instances = {}
    instance_limit = 0

    def __call__(cls, *args, **kwargs):
        if len(cls._instances.get(cls, [])) < cls.instance_limit:
            instance = super().__call__(*args, **kwargs)
            cls._instances.setdefault(cls, []).append(instance)
            return instance
        return cls._instances[cls][0]  # Return the first instance if limit is reached
#

In [ ]:
class LC(metaclass=LimitedInstances):
    def __init__(self, x):
        self.value = x
obj = LC(5)

KeyError: <class '__main__.LC'>

In [ ]:
class LimitedSingleton(metaclass=LimitedInstances):
    instance_limit = 1

    def __init__(self, x):
        self.value = x

x1 = LimitedSingleton("x1")
x2 = LimitedSingleton("x2")
print(x1, x2)
print(x1.value, x2.value)
print(x1 is x2)

<__main__.LimitedSingleton object at 0x79b0a42e1540> <__main__.LimitedSingleton object at 0x79b0a42e1540>
x1 x1
True


In [ ]:
# 'Tripleton' --  allow up to 3 instances
class TripletonExample(LimitedSingleton):
    instance_limit = 3

In [ ]:
a = TripletonExample("o1")
b = TripletonExample("o2")
c = TripletonExample("o3")
d = TripletonExample("o4")

print(a.value, b.value, c.value, d.value)

o1 o2 o3 o1


In [ ]:
print(a is b, b is c, c is d, a is d)

False False False True


In [ ]:
LimitedInstances._instances

{__main__.LimitedSingleton: [<__main__.LimitedSingleton at 0x79b0a42e1540>],
 __main__.TripletonExample: [<__main__.TripletonExample at 0x79b0a42e1f30>,
  <__main__.TripletonExample at 0x79b0a42e1000>]}

Інший варіант реалізації:

In [ ]:
def hook(nm):
    def func(*a, **k):
        res = getattr(type, nm)(*a, **k)
        print(nm, ':', a, ';', k, '--->', res)
        return res
    return func
verbose_type = type('TP', (type,), {nm: hook(nm) for nm in ['__new__', '__init__', '__call__']})

In [ ]:
class B(metaclass=verbose_type):
    pass

__new__ : (<class '__main__.TP'>, 'B', (), {'__module__': '__main__', '__qualname__': 'B'}) ; {} ---> <class '__main__.B'>
__init__ : (<class '__main__.B'>, 'B', (), {'__module__': '__main__', '__qualname__': 'B'}) ; {} ---> None


In [ ]:
b1 = B()
b2 = B()

__call__ : (<class '__main__.B'>,) ; {} ---> <__main__.B object at 0x79b0819ce3e0>
__call__ : (<class '__main__.B'>,) ; {} ---> <__main__.B object at 0x79b0819cf2b0>


In [ ]:
class C(B):
    pass

__new__ : (<class '__main__.TP'>, 'C', (<class '__main__.B'>,), {'__module__': '__main__', '__qualname__': 'C'}) ; {} ---> <class '__main__.C'>
__init__ : (<class '__main__.C'>, 'C', (<class '__main__.B'>,), {'__module__': '__main__', '__qualname__': 'C'}) ; {} ---> None


In [ ]:
c1 = C()
c2 = C()

__call__ : (<class '__main__.C'>,) ; {} ---> <__main__.C object at 0x79b0819cf910>
__call__ : (<class '__main__.C'>,) ; {} ---> <__main__.C object at 0x79b0819cc9d0>


In [ ]:
class Multiton(type):
    _storage = {}
    def __new__(cls, name, bases, dct, max_instances = 0):
        reslt = super().__new__(cls, name, bases, dct)
        reslt._num_objs = 0
        print(f'__new__ in {cls}; reslt = {reslt}')

        original_new = reslt.__new__

        def fake_new(objclass, *args, **kwargs):
            print(f'__new__ called in {reslt}, with params', end=' ')
            print(objclass, objclass._num_objs,  max_instances)
            if objclass._num_objs < max_instances:
                instance = original_new(objclass, *args, **kwargs)
                objclass._num_objs += 1
                return instance
            else:
                raise Exception(f"Cannot create {cls.__name__}")

        reslt.__new__ = fake_new # !!!
        return reslt

class A1(metaclass=Multiton, max_instances=1):
  pass

__new__ in <class '__main__.Multiton'>; reslt = <class '__main__.A1'>


In [ ]:
a1 = A1()
a2 = A1()

__new__ called in <class '__main__.A1'>, with params <class '__main__.A1'> 0 1
__new__ called in <class '__main__.A1'>, with params <class '__main__.A1'> 1 1


Exception: Cannot create Multiton

In [ ]:
class B3(metaclass=Multiton, max_instances=3):
  pass
b1 = B3()
b2 = B3()
b3 = B3()

__new__ in <class '__main__.Multiton'>; reslt = <class '__main__.B3'>
__new__ called in <class '__main__.B3'>, with params <class '__main__.B3'> 0 3
__new__ called in <class '__main__.B3'>, with params <class '__main__.B3'> 1 3
__new__ called in <class '__main__.B3'>, with params <class '__main__.B3'> 2 3


In [ ]:
dir(B3)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_num_objs']

In [ ]:
B3._num_objs

3

In [ ]:
b4 = B3()

__new__ called in <class '__main__.B3'>, with params <class '__main__.B3'> 3 3


Exception: Cannot create Multiton

&nbsp;

---

Бонус -- multiple inheritance for metaclasses:

In [ ]:
class Meta1(type):
    def __init__(cls, name, bases, dct):
        super().__init__(name, bases, dct)
        print(f"Meta1 for {name}")

class Meta2(type):
    def __init__(cls, name, bases, dct):
        super().__init__(name, bases, dct)
        print(f"Meta2 for {name}")

class CombinedMeta(Meta1, Meta2):
    pass

In [ ]:
class MyClass(metaclass=CombinedMeta):
    pass

Meta2 for MyClass
Meta1 for MyClass
